In [1]:
#Import necessary libraries
import os
import ee
import geemap
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import statsmodels.api as sm
import pandas as pd
from IPython.display import HTML, display
import random
import json
import time
num_seed=30
random.seed(num_seed)

In [2]:
#Initialize earth engine
ee.Initialize()


In [3]:
#Define functions for mapping MapBiomas and simplifying the legend
coverage_palette =  ['ffffff', '129912', '1f4423', '006400', '00ff00', '687537', '76a5af', '29eee4', 
                     '77a605', '935132', 'bbfcac', '45c2a5', 'b8af4f', 'f1c232', 'ffffb2', 'ffd966', 
                     'f6b26b', 'f99f40', 'e974ed', 'd5a6bd', 'c27ba0', 'fff3bf', 'ea9999', 'dd7e6b', 
                     'aa0000', 'ff99ff', '0000ff', 'd5d5e5', 'dd497f', 'b2ae7c', 'af2a2a', '8a2be2', 
                     '968c46', '0000ff', '4fd3ff']


simple_palette = ['129912','BBFCAC','FFFFB2','EA9999','0000FF','D5D5E5']
statesViz = {'min': 0, 'max': 34, 'palette': coverage_palette};
simpleStatesViz = {'min': 1, 'max': 6, 'palette': simple_palette};

change_detection_palette = ['df07b5','0741df']
changeDetectionViz = {'min': 0, 'max': 1, 'palette': change_detection_palette};

#Load in mapbiomas
mapbiomas_states=ee.Image('projects/mapbiomas-workspace/public/collection4_1/mapbiomas_collection41_integration_v1')
states_mask = mapbiomas_states.mask()

#Define function to convert hierarchical legend to simplest form
def simplify_legend(bandName):
    simplify = mapbiomas_states.expression(
        '(b0 >=1)  && (b0<10) ? 1 :'+
        '((b0>=10) && (b0<14)) || (b0==32) || (b0==29) ? 2 :'+
        '((b0>=18) && (b0<22)) || ((b0>=14)&&(b0<16)) ? 3 :'+
        '((b0>=22) && (b0<26)) || (b0==30) ? 4 :'+
        '(b0==26) || (b0==33) || (b0==31) ? 5 : 6', 
        {
          'b0': mapbiomas_states.select([bandName])
        })
    simplify = simplify.select(['constant'],[bandName])
    return simplify

#Select bands we are interested in
bandList = ['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 
             'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 
             'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 
             'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 
             'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 
             'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 
             'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 
             'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 
             'classification_2017', 'classification_2018']
bandsEEList = ee.List(bandList) 
states_simple = ee.ImageCollection(bandsEEList.map(simplify_legend)).toBands()
states_simple = states_simple.select(states_simple.bandNames(),bandsEEList)
states_simple = states_simple.updateMask(states_mask)
states_simple = states_simple.set(ee.Dictionary({'min_value':1,'max_value':5}))
#states_simple is now an iamge where each band corresponds to the land cover class for the band name year

#Map one year to check it out!
Map1 = geemap.Map(center=[-9,-51], zoom=4)
Map1.addLayer(mapbiomas_states.select('classification_2018'),statesViz,name='Original MapBiomas')
Map1.addLayer(states_simple.select('classification_2018'),simpleStatesViz,name='Simplified MapBiomas')
display(Map1)


Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [4]:
#Save 30 meter projection
#Save 30 meter projection
projection_30m = mapbiomas_states.projection().getInfo()
crs = projection_30m.get('crs')
crsTransform = projection_30m.get('transform')
scale = mapbiomas_states.projection().nominalScale().getInfo()
print(crs, crsTransform,scale)


EPSG:4326 [0.0002694945852358564, 0, -74.54381924374933, 0, -0.0002694945852358564, 6.792611020869763] 29.999999999999996


In [5]:
b_names = mapbiomas_states.bandNames().getInfo()

In [6]:
# reduce frequency histograms from one year to the next
# then parse reducer output json to dataframes and add each subsequent comparison
df = pd.DataFrame()
for i,x in enumerate(b_names[:-1]):
    year0 = mapbiomas_states.select(b_names[i])
    year1 = mapbiomas_states.select(b_names[i+1])
    hist = year0.addBands(year1).reduceRegion(
        reducer=ee.Reducer.frequencyHistogram().group(groupField=1,groupName='class'),
        bestEffort=True,
        scale=30)
    hist_table = hist.getInfo()
    df_tm = pd.io.json.json_normalize(hist_table['groups']).set_index('class').fillna(0)
    df = df.add(df_tm, fill_value=0)
    time.sleep(0.5)

In [7]:
df.columns = [x.replace('histogram.','') for x in df.columns] # remove the 'histogram.' prefix from json io parse
cols = sorted(df.columns.values,key=int) # sort string numbers by pretending they're real numbers
df = df[cols].fillna(0) # nans are actually 0 in this case
df_subclass = df.copy()
np.fill_diagonal(df_subclass.values, 0)

In [8]:
print('Transition probability matrix for default classes (from row class to column class):')
df_subclass.divide(df_subclass.sum(axis=1),axis=0).style.format('{:,.2%}').background_gradient(cmap='BuPu',axis=1)

Transition probability matrix for default classes (from row class to column class):


,0,3,4,5,9,11,12,13,15,19,20,21,23,24,25,29,30,31,32,33
class,,,,,,,,,,,,,,,,,,,,
0,0.00%,22.17%,4.68%,24.33%,0.00%,0.00%,0.00%,0.01%,0.01%,0.06%,0.00%,8.22%,17.02%,3.35%,0.09%,0.00%,0.00%,10.93%,1.63%,7.51%
3,0.00%,0.00%,15.39%,0.40%,1.98%,0.95%,5.35%,6.48%,46.96%,6.56%,0.70%,7.92%,0.03%,0.40%,0.10%,0.08%,0.00%,0.00%,0.02%,6.67%
4,0.00%,23.86%,0.00%,0.03%,0.59%,0.50%,15.00%,0.01%,53.12%,2.56%,0.11%,2.57%,0.05%,0.15%,0.37%,0.02%,0.00%,0.00%,0.02%,1.03%
5,0.45%,47.20%,2.55%,0.00%,0.69%,0.00%,1.77%,4.96%,8.07%,0.10%,0.16%,8.30%,1.50%,1.63%,0.18%,0.00%,0.00%,0.58%,3.88%,17.98%
9,0.00%,42.10%,8.48%,0.11%,0.00%,0.05%,6.32%,0.70%,31.69%,4.64%,0.55%,4.70%,0.08%,0.12%,0.10%,0.02%,0.00%,0.00%,0.00%,0.35%
11,0.00%,36.38%,12.59%,0.00%,0.03%,0.00%,26.38%,0.00%,8.26%,5.33%,0.00%,0.00%,0.28%,0.03%,0.06%,0.00%,0.00%,0.00%,0.00%,10.67%
12,0.00%,15.82%,34.23%,0.05%,0.89%,2.40%,0.00%,0.00%,20.53%,22.50%,0.31%,0.95%,0.23%,0.25%,0.61%,0.04%,0.01%,0.03%,0.07%,1.09%
13,0.00%,78.05%,0.07%,0.48%,0.71%,0.00%,0.00%,0.00%,5.30%,0.43%,0.06%,2.54%,0.32%,0.22%,0.05%,0.00%,0.02%,0.00%,0.22%,11.53%
15,0.00%,44.19%,30.49%,0.04%,0.58%,0.12%,4.59%,0.27%,0.00%,6.37%,1.52%,9.95%,0.04%,0.46%,0.14%,0.03%,0.00%,0.00%,0.00%,1.19%


In [9]:
df_subclass.sum(axis=1)

class
0         64.066667
3     344418.823529
4     239243.380392
5       3057.952941
9      28042.549020
11      9970.470588
12    107603.964706
13     28634.549020
15    622469.670588
19    192412.858824
20     46569.847059
21    127644.827451
23      1428.317647
24     12301.015686
25      4114.941176
29      1031.039216
30        92.000000
31       205.180392
32       501.564706
33     46156.772549
dtype: float64

In [10]:
# Double check: is this the correct class simplification?
simplify_dict = {
    0:6,
    1:1,
    2:1,
    3:1,
    4:1,
    5:1,
    6:1,
    7:1,
    8:1,
    9:1,
    10:2,
    11:2,
    12:2,
    13:2,
    14:3,
    15:3,
    16:6,
    17:6,
    18:3,
    19:3,
    20:3,
    21:3,
    22:4,
    23:4,
    24:4,
    25:4,
    26:5,
    27:6,
    28:6,
    29:2,
    30:4,
    31:5,
    32:2,
    33:5
}

In [11]:
g = df.copy()
g['from ⬇️'] = g.index.map(simplify_dict).values
df_simple = g.groupby('from ⬇️').sum().T
df_simple['to ➡️'] = df_simple.index.astype(int).map(simplify_dict).values
df_simple = df_simple.groupby('to ➡️').sum().T
np.fill_diagonal(df_simple.values, 0)
print('Simplified transition probability matrix:')
df_simple.divide(df_simple.sum(axis=1),axis=0).style.format('{:,.2%}').background_gradient(cmap='BuPu',axis=1)

Simplified transition probability matrix:


to ➡️,1,2,3,4,5,6
from ⬇️,,,,,,
1,0.00%,17.50%,76.33%,0.71%,5.45%,0.01%
2,58.39%,0.00%,36.60%,1.02%,3.98%,0.00%
3,86.11%,11.04%,0.00%,1.22%,1.63%,0.00%
4,26.33%,11.10%,58.70%,0.00%,3.80%,0.06%
5,62.19%,13.30%,23.22%,1.26%,0.00%,0.03%
6,51.18%,1.64%,8.28%,20.46%,18.44%,0.00%


In [12]:
df_simple.sum(axis=1)

from ⬇️
1    479246.133333
2    142184.552941
3    648552.203922
4     17582.725490
5     46320.098039
6        64.066667
dtype: float64